In [7]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
from scipy import stats

In [2]:
class Linear_Bandit():
    '''
    stochastic linear (contextual) bandit environment
    rewards are Gaussian
    ============================================
    k: number of arms (int)
    n: length of horizon (int)
    sigma: reward is sigma^2-SubGaussian, list of positive float with length k
    beta: linear coefficient for mean reward for each arm, true parameter for linear bandit
          k by d(or 1 by d) numpy array for user-defined values.
    random_context: True if contexts are stochastic/generated from some distribution,
                    False if contexts are deterministic/fixed.
    gen_context: function to generate d-dimensional context for all arms, output k by d numpy array
    '''
    def  __init__(self, k, n, beta, sigma, random_context = True, gen_context = None):
        
        self.k = k                                  # number of arms
        self.n = n                                  # length of horizon
        self.sigma = sigma                          # SubGaussian constants
        self.beta = np.array(beta)                  # linear coefficient, parameter for linear bandit
        self.d = beta.shape[1]                      # dimension of context                 
        
        if gen_context is None:
            print("error: please specify a funtion for generating context")
            
        # make tables
        if beta.shape[0] == 1:
            ## This is the case that true coefficient is shared by arms
            betas = np.repeat(beta, k, axis = 0)
        else:
            ## This is the case that coefficients are different among arms(linear bandit with covariates)
            betas = beta
        
        d = beta.shape[1]
        reward_table = np.zeros(k*n).reshape(n ,k)
        context_table = np.zeros(d*n*k).reshape(n, k, d)
        mu_table = np.zeros(k*n).reshape(n, k)
        if random_context:
            ## random context, context is generated at each time
            for i in range(n):
                c = gen_context(k = k, d = d)
                context_table[i,:,:] = c
                for j in range(k):
                    mu_table[i,j] = np.inner(betas[j,:], c[j,:])
                reward_table[i,:] = stats.multivariate_normal.rvs(mean = mu_table[i,:], cov = np.diag(sigma**2))
        else:
            ## fixed context, context is fixed along time
            c = gen_context(k = k, d = d)
            mu = np.zeros(k).reshape(k)
            for j in range(k):
                mu[j] = np.inner(betas[j,:], c[j,:])
            for i in range(n):
                context_table[i,:,:] = c
                mu_table[i,:] = mu
                reward_table[i,:] = stats.multivariate_normal.rvs(mean = mu, cov = np.diag(sigma**2))
        
        self.rewards = reward_table                       # reward table, n by k array
        self.contexts = context_table                     # context table, n by k by d array
        self.mus = mu_table                               # mu table, n by k array
        self.random_arms = np.random.randint(1, k+1, n)   # random arms sequence with length n for arm-independent context 

    def pull(self, a, t):
        '''
        pull arm/take action and observe reward
        ============================================
        INPUT
            a: action
            t: time
        ============================================
        OUPUT
            r: reward
        '''
        r = self.rewards[t,a-1]
        return r
        
    
    def get_context(self, t, all_arm = True):
        '''
        get context
        ============================================
        INPUT
            t: time
            all_arm: True return k by d array for k contexts; False return one d by 1 context randomly
        ============================================
        OUPUT
            c: context, k by d or d by 1
        '''
        if not all_arm:
            c = self.contexts[t, self.random_arms[t]-1, :].reshape(self.d, 1) 
        else:
            c = self.contexts[t, :, :]
        return c
    
    def regret(self, a, t):
        '''
        regret for current action
        ============================================
        INPUT
            a: action
            t: time
        ============================================
        OUPUT
            regret: regret
        '''
        mu = self.mus[t,:]
        mu_best = max(mu)
        regret = mu_best - mu[a-1]
        return regret
    
    def beta_sharing(self):
        '''
        indicator for beta sharing among arms
        ============================================
        INPUT
        ============================================
        OUPUT
            sharing: True if beta is same among arms
        '''
        return self.beta.shape[0] == 1

In [20]:
def Linear_ReBoot_G(env, lam, weight_sd, coefficient_sharing = True):
    '''
    Gaussian Residual Boostrap Exploration assuming linear contextual bandit
    This is a linear bandit algorithm
    ============================================
    INPUTS
        env: stochastic bandit environment
        lam: regularization parameter
        weight_sd: standard deviation of residual bootstrap weights
        coefficient sharing: True if assuming true coefficient is same among arms
    ============================================
    OUPUTS 
        R: reward sequence, list with length n
        A is action sequence, list with length n
        regret: regret sequence, list with length n
    '''
    # set up
    n = env.n
    K = env.k
    d = env.d
    lam = lam + 1e-20
    regret = [0]
    A = []
    R = []
    
    if not coefficient_sharing:
        beta_est = np.zeros(d*K).reshape(d, K)
        V_est = [np.identity(d)*lam for i in range(K)]
        g = [np.zeros(d).reshape(d,1) for i in range(K)]
        arm_count = np.zeros(K)
        Sum1_by_A = np.zeros(K)
        Sum2_by_A = np.zeros(K)
        Y_by_A = [np.empty((0,1))]*K
        X_by_A = [np.empty((0,d))]*K
        
        # temporary liat/array
        mu_est = np.zeros(K)
        
        # pull each arm once
        for t in range(1, K+1):
            a_t = t
            c_t = env.get_context(t, all_arm = False)
            r_t = env.pull(a_t, t)
            A.append(a_t)
            R.append(r_t)
            Y_by_A[a_t - 1] = np.append(Y_by_A[a_t - 1], np.array(r_t).reshape(1, 1), axis = 0)
            X_by_A[a_t - 1] = np.append(X_by_A[a_t - 1], np.array(c_t).reshape(1, d), axis = 0)
            ##incremental update
            V_est[a_t - 1] = V_est[a_t - 1] + np.matmul(c_t, c_t.T)
            g[a_t - 1] = g[a_t - 1] + r_t * c_t.reshape(d,1)
            Sum1_by_A[a_t - 1] = Sum1_by_A[a_t - 1] + r_t
            Sum2_by_A[a_t - 1] = Sum2_by_A[a_t - 1] + r_t**2
            arm_count[a_t - 1] = arm_count[a_t - 1] + 1
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
            
        # ReBoot loop
        for t in range(K+1, n):
            ## LSE update
            X = X_by_A[a_t - 1]
            Y = Y_by_A[a_t - 1]
            beta_est[:,a_t - 1] = (np.linalg.inv(V_est[a_t - 1]) @ g[a_t - 1]).reshape(d)

            
            ## ReBoot exploration
            c_t = env.get_context(t, all_arm = False)
            mu_hat = np.matmul(c_t.T, beta_est).reshape(K)
            Sigma_diag = (Sum2_by_A + arm_count * mu_hat * mu_hat - 2 * mu_hat * mu_hat * Sum1_by_A)/(arm_count*arm_count) 
            mu_est = stats.multivariate_normal.rvs(size = 1, mean = mu_hat, cov = np.diag(weight_sd**2 * Sigma_diag))
            
            ## pull arm
            a_t = np.argmax(mu_est) + 1
            r_t = env.pull(a_t, t)
            A.append(a_t)
            R.append(r_t)
            Y_by_A[a_t - 1] = np.append(Y_by_A[a_t - 1], np.array(r_t).reshape(1, 1), axis = 0)
            X_by_A[a_t - 1] = np.append(X_by_A[a_t - 1], np.array(c_t).reshape(1, d), axis = 0)
            
            ## incremental update
            V_est[a_t - 1] = V_est[a_t - 1] + np.matmul(c_t, c_t.T)
            g[a_t - 1] = g[a_t - 1] + r_t * c_t.reshape(d,1)
            Sum1_by_A[a_t - 1] = Sum1_by_A[a_t - 1] + r_t
            Sum2_by_A[a_t - 1] = Sum2_by_A[a_t - 1] + r_t**2
            arm_count[a_t - 1] = arm_count[a_t - 1] + 1

            ## compute regret
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
            
    else:
        beta_est = np.zeros(d).reshape(d, 1)
        V_est = np.identity(d)*lam
        g = np.zeros(d).reshape(d,1)
        arm_count = np.zeros(K)
        Sum1_by_A = np.zeros(K)
        Sum2_by_A = np.zeros(K)
        Y = np.empty((0,1))
        X = np.empty((0,d))
        
        # pull each arm once
        for t in range(1, K+1):
            a_t = t
            r_t = env.pull(a_t, t)
            c_K = env.get_context(t, True)
            c_t = c_K[a_t - 1,:].reshape(d,1)
            A.append(a_t)
            R.append(r_t)
            Y = np.append(Y, np.array(r_t).reshape(1, 1), axis = 0)
            X = np.append(X, np.array(c_t).reshape(1, d), axis = 0)
            V_est = V_est + np.matmul(c_t, c_t.T)
            g = g + r_t * c_t.reshape(d,1)
            Sum1_by_A[a_t - 1] = Sum1_by_A[a_t - 1] + r_t
            Sum2_by_A[a_t - 1] = Sum2_by_A[a_t - 1] + r_t**2
            arm_count[a_t - 1] = arm_count[a_t - 1] + 1
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
            
        # ReBoot loop
        for t in range(K+1, n):
            ## LSE update
            beta_est = np.linalg.inv(V_est) @ g
            
            ## ReBoot exploration
            c_K = env.get_context(t, True)
            mu_hat = np.matmul(c_K, beta_est).reshape(K)
            Sigma_diag = (Sum2_by_A + arm_count * mu_hat * mu_hat - 2 * mu_hat * mu_hat * Sum1_by_A)/(arm_count*arm_count) 
            mu_est = stats.multivariate_normal.rvs(size = 1, mean = mu_hat, cov = np.diag(weight_sd**2 * Sigma_diag))
            
            ## pull arm
            a_t = np.argmax(mu_est) + 1
            r_t = env.pull(a_t, t)
            c_t = c_K[a_t - 1,:].reshape(d,1)
            A.append(a_t)
            R.append(r_t)
            Y = np.append(Y, np.array(r_t).reshape(1, 1), axis = 0)
            X = np.append(X, np.array(c_t).reshape(1, d), axis = 0)
            
            # incremental update
            V_est = V_est + np.matmul(c_t, c_t.T)
            g = g + r_t * c_t.reshape(d,1)
            Sum1_by_A[a_t - 1] = Sum1_by_A[a_t - 1] + r_t
            Sum2_by_A[a_t - 1] = Sum2_by_A[a_t - 1] + r_t**2
            arm_count[a_t - 1] = arm_count[a_t - 1] + 1

            ## compute regret
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
            
    return R, A, regret

In [45]:
def Linear_TS_G(env, tau, coefficient_sharing = True):
    '''
    Linear Thompson Sampling with Gaussian prior N(0, (tau^2)*I)
    Same piror for each arm
    This is a linear bandit algorithm
    ============================================
    INPUTS
        env: stochastic linear bandit environment
        tau: std in prior (positive float)
        coefficient sharing: True if assuming true coefficient is same among arms
    ============================================
    OUPUTS 
        R: reward sequence, list with length n
        A is action sequence, list with length n
        regret: regret sequence, list with length n
    ''' 
    # set up
    n = env.n
    K = env.k
    d = env.d
    regret = [0]
    A = []
    R = []
    
    if not coefficient_sharing:
        beta_mean = [np.zeros(d) for i in range(K)]
        beta_V = [np.identity(d)*(1/tau**2) for i in range(K)]
        beta_sigma = [np.identity(d)*tau**2 for i in range(K)]
        beta_g = [np.zeros(d).reshape(d,1) for i in range(K)]
        sigma_est = np.ones(K)
        Y_by_A = [np.empty((0,1))]*K
        X_by_A = [np.empty((0,d))]*K


        # temporary liat/array
        beta_est = np.zeros(K*d).reshape(K,d)

        # Thompson Sampling loop
        for t in range(1, n):
            ## estimation
            c_t = env.get_context(t, all_arm = False)
            for j in range(K):
                beta_est[j,:]  = stats.multivariate_normal.rvs(mean = beta_mean[j].reshape(d), cov = (sigma_est[j]**2)*beta_sigma[j])
            c_t = np.array(c_t).reshape(d, 1)
            mu_est = np.matmul(beta_est, c_t).reshape(K)

            ## pull arm
            a_t = np.argmax(mu_est) + 1
            r_t = env.pull(a_t, t)
            A.append(a_t)
            R.append(r_t)
            Y_by_A[a_t - 1] = np.append(Y_by_A[a_t - 1], np.array(r_t).reshape(1, 1), axis = 0)
            X_by_A[a_t - 1] = np.append(X_by_A[a_t - 1], np.array(c_t).reshape(1, d), axis = 0)

            ## update
            X = X_by_A[a_t - 1]
            Y = Y_by_A[a_t - 1]
            sigma_est[a_t - 1] = np.std(Y)
            beta_V[a_t - 1] = beta_V[a_t - 1] + np.matmul(c_t, c_t.T)
            beta_sigma[a_t - 1] = np.linalg.inv(beta_V[a_t - 1])
            beta_g[a_t - 1] = beta_g[a_t - 1] + r_t * c_t.reshape(d,1)
            beta_mean[a_t - 1] = np.matmul(beta_sigma[a_t - 1], beta_g[a_t - 1])


            ## compute regret
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
    
    else:
        beta_mean = np.zeros(d)
        beta_V = np.identity(d)*(1/tau**2)
        beta_sigma = np.identity(d)*tau**2
        beta_g = np.zeros(d).reshape(d,1)
        sigma_est = np.ones(1)
        Y = np.empty((0,1))
        X = np.empty((0,d))

        # pull each arm once
        for t in range(1, K+1):
            a_t = t
            r_t = env.pull(a_t, t)
            c_K = env.get_context(t, True)
            c_t = c_K[a_t - 1,:].reshape(d,1)
            A.append(a_t)
            R.append(r_t)
            Y = np.append(Y, np.array(r_t).reshape(1, 1), axis = 0)
            X = np.append(X, np.array(c_t).reshape(1, d), axis = 0)
            sigma_est = np.std(Y)
            beta_V = beta_V + np.matmul(c_t, c_t.T)
            beta_sigma = np.linalg.inv(beta_V)
            beta_g = beta_g + r_t * c_t.reshape(d,1)
            beta_mean = np.matmul(beta_sigma, beta_g)
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
        
        # Thompson Sampling loop
        for t in range(K+1, n):            
            ## estimation
            c_K = env.get_context(t, True)
            beta_est  = stats.multivariate_normal.rvs(mean = beta_mean.reshape(d), cov = (sigma_est**2)*beta_sigma)
            mu_est = np.matmul(c_K, beta_est).reshape(K)
        
            ## pull arm
            a_t = np.argmax(mu_est) + 1
            r_t = env.pull(a_t, t)
            c_t = c_K[a_t - 1,:].reshape(d,1)
            A.append(a_t)
            R.append(r_t)
            Y = np.append(Y, np.array(r_t).reshape(1, 1), axis = 0)
            X = np.append(X, np.array(c_t).reshape(1, d), axis = 0)

            ## update
            sigma_est = np.std(Y)
            beta_V = beta_V + np.matmul(c_t, c_t.T)
            beta_sigma = np.linalg.inv(beta_V)
            beta_g = beta_g + r_t * c_t.reshape(d,1)
            beta_mean = np.matmul(beta_sigma, beta_g)

            ## compute regret
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
        
    return R, A, regret

In [46]:
def Linear_TS_IG(env, tau, alpha, coefficient_sharing = True):
    '''
    Linear Thompson Sampling with priors N(0, (tau^2)*I) and IG(alpha, alpha)
    Same piror for each arm
    This is a linear bandit algorithm
    ============================================
    INPUTS
        env: stochastic linear bandit environment
        tau: std in Gaussian prior (positive float)
        alpha: parameter in Inverse Gamma prior
        coefficient sharing: True if assuming true coefficient is same among arms
    ============================================
    OUPUTS 
        R: reward sequence, list with length n
        A is action sequence, list with length n
        regret: regret sequence, list with length n
    ''' 
    # set up
    n = env.n
    K = env.k
    d = env.d
    regret = [0]
    A = []
    R = []
    
    if not coefficient_sharing:
        beta_mean = [np.zeros(d) for i in range(K)]
        beta_V = [np.identity(d)*(1/tau**2) for i in range(K)]
        beta_sigma = [np.identity(d)*tau**2 for i in range(K)]
        beta_g = [np.zeros(d).reshape(d,1) for i in range(K)]
        eta_a = np.ones(K)*alpha
        eta_b = np.ones(K)*alpha
        sigma_est = np.ones(K)
        Y_by_A = [np.empty((0,1))]*K
        X_by_A = [np.empty((0,d))]*K


        # temporary liat/array
        beta_est = np.zeros(K*d).reshape(K,d)

        # Thompson Sampling loop
        for t in range(1, n):
            c_t = env.get_context(t, False)
            for j in range(K):
                sigma_est[j] = scipy.stats.invgamma.rvs(a=eta_a[j], scale=eta_b[j])
                beta_est[j,:]  = stats.multivariate_normal.rvs(mean = beta_mean[j].reshape(d), cov = sigma_est[j]*beta_sigma[j])
            c_t = np.array(c_t).reshape(d, 1)
            mu_est = np.matmul(beta_est, c_t).reshape(K)

            ## pull arm
            a_t = np.argmax(mu_est) + 1
            r_t = env.pull(a_t, t)
            A.append(a_t)
            R.append(r_t)
            Y_by_A[a_t - 1] = np.append(Y_by_A[a_t - 1], np.array(r_t).reshape(1, 1), axis = 0)
            X_by_A[a_t - 1] = np.append(X_by_A[a_t - 1], np.transpose(c_t), axis = 0)

            ## update
            X = X_by_A[a_t - 1]
            Y = Y_by_A[a_t - 1]
            beta_V[a_t - 1] = beta_V[a_t - 1] + np.matmul(c_t, c_t.T)
            beta_sigma[a_t - 1] = np.linalg.inv(beta_V[a_t - 1])
            beta_g[a_t - 1] = beta_g[a_t - 1] + r_t * c_t.reshape(d,1)
            beta_mean[a_t - 1] = np.matmul(beta_sigma[a_t - 1], beta_g[a_t - 1])
            eta_a[a_t - 1] = eta_a[a_t - 1] + 1/2
            eta_b[a_t - 1] = alpha + 0.5*np.matmul(beta_g[a_t - 1].T, beta_mean[a_t - 1])

            ## compute regret
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)

    
    else:
        beta_mean = np.zeros(d)
        beta_V = np.identity(d)*(1/tau**2)
        beta_sigma = np.identity(d)*tau**2
        beta_g = np.zeros(d).reshape(d,1)
        eta_a = np.ones(1)*alpha
        eta_b = np.ones(1)*alpha
        sigma_est = np.ones(1)
        Y = np.empty((0,1))
        X = np.empty((0,d))
        X_K = np.zeros(d*K).reshape(K,d)
        
        # pull each arm once
        for t in range(1, K+1):
            a_t = t
            r_t = env.pull(a_t, t)
            c_K = env.get_context(t, True)
            c_t = c_K[a_t - 1,:].reshape(d,1)
            A.append(a_t)
            R.append(r_t)
            Y = np.append(Y, np.array(r_t).reshape(1, 1), axis = 0)
            X = np.append(X, np.array(c_t).reshape(1, d), axis = 0)
            beta_V = beta_V + np.matmul(c_t, c_t.T)
            beta_sigma = np.linalg.inv(beta_V)
            beta_g = beta_g + r_t * c_t.reshape(d,1)
            beta_mean = np.matmul(beta_sigma, beta_g)
            eta_a = eta_a + 1/2
            eta_b = alpha + 0.5*np.matmul(beta_g.T, beta_mean)
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
        
        # Thompson Sampling loop
        for t in range(K+1, n):         
            ## estimation
            c_K = env.get_context(t, True)
            sigma_est = scipy.stats.invgamma.rvs(a=eta_a, scale=eta_b)
            beta_est  = stats.multivariate_normal.rvs(mean = beta_mean.reshape(d), cov = (sigma_est**2)*beta_sigma)
            mu_est = np.matmul(c_K, beta_est).reshape(K)
        
            ## pull arm
            a_t = np.argmax(mu_est) + 1
            r_t = env.pull(a_t, t)
            c_t = c_K[a_t - 1,:].reshape(d,1)
            A.append(a_t)
            R.append(r_t)
            Y = np.append(Y, np.array(r_t).reshape(1, 1), axis = 0)
            X = np.append(X, np.array(c_t).reshape(1, d), axis = 0)
            
            # update
            beta_V = beta_V + np.matmul(c_t, c_t.T)
            beta_sigma = np.linalg.inv(beta_V)
            beta_g = beta_g + r_t * c_t.reshape(d,1)
            beta_mean = np.matmul(beta_sigma, beta_g)
            eta_a = eta_a + 1/2
            eta_b = alpha + 0.5*np.matmul(beta_g.T, beta_mean)
            
            ## compute regret
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
        
    return R, A, regret

In [47]:
def Linear_GIRO(env, a, lam, R_upper, R_lower, coefficient_sharing = True):
    '''
    Garbage In, Reward Out Algorithm
    Boostrap exploration for bounded reward
    This is a linear bandit algorithm
    ============================================
    INPUTS
        env: stochastic bandit environment
        a: number of postive/negative pseudo rewards per time unit (int)
        lam: regularization parameter 
        R_upper: upper bound for reward  
        R_lower: lower bound for reward
        coefficient sharing: True if assuming true coefficient is same among arms
    ============================================
    OUPUTS 
        R: reward sequence, list with length n
        A is action sequence, list with length n
        regret: regret sequence, list with length n
    '''
    # set up
    n = env.n
    K = env.k
    d = env.d
    lam = lam + 1e-20
    regret = [0]
    A = []
    R = []
    
    if not coefficient_sharing:
        beta_est = [np.zeros(d) for i in range(K)]
        V_est = [np.identity(d)*lam for i in range(K)]
        Y_by_A = [np.empty((0,1))]*K
        X_by_A = [np.empty((0,d))]*K

        # temporary liat/array
        mu_est = np.zeros(K)

        # GIRO loop
        for t in range(1, n):
            c_t = env.get_context(t, False)
            c_t = np.array(c_t).reshape(d, 1)
            for k in range(K):
                s = len(Y_by_A[k])
                ## Boostrapping
                idx = np.random.choice(s,s)
                X = X_by_A[k][idx,:]
                Y = Y_by_A[k][idx,:]
                V_est[k] = np.linalg.inv(np.matmul(np.transpose(X), X) + np.identity(d)*lam)
                beta_est[k] = np.matmul(V_est[k], np.matmul(np.transpose(X), Y))
                mu_est[k] = np.matmul(np.transpose(beta_est[k]), c_t)

            ## pull arm
            a_t = np.argmax(mu_est) + 1
            r_t = env.pull(a_t, t)
            A.append(a_t)
            R.append(r_t)
            Y_by_A[a_t - 1] = np.append(Y_by_A[a_t - 1], np.array(r_t).reshape(1, 1), axis = 0)
            X_by_A[a_t - 1] = np.append(X_by_A[a_t - 1], np.transpose(c_t), axis = 0)

            ## pseudo rewards
            for i in range(a):
                Y_by_A[a_t - 1] = np.append(Y_by_A[a_t - 1], np.array(R_upper).reshape(1, 1), axis = 0)
                X_by_A[a_t - 1] = np.append(X_by_A[a_t - 1], np.transpose(c_t), axis = 0)
                Y_by_A[a_t - 1] = np.append(Y_by_A[a_t - 1], np.array(R_lower).reshape(1, 1), axis = 0)
                X_by_A[a_t - 1] = np.append(X_by_A[a_t - 1], np.transpose(c_t), axis = 0)

            ## compute regret
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
    
    else:
        beta_est = np.zeros(d)
        V_est = np.identity(d)*lam
        Y = np.empty((0,1))
        X = np.empty((0,d))
        
        # pull each arm once
        for t in range(1, K+1):
            a_t = t
            c_K = env.get_context(t, True)
            c_t = c_K[a_t - 1,:].reshape(d,1)
            r_t = env.pull(a_t, t)
            A.append(a_t)
            R.append(r_t)
            Y = np.append(Y, np.array(r_t).reshape(1, 1), axis = 0)
            X = np.append(X, np.array(c_t).reshape(1, d), axis = 0)
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
            
        # GIRO loop
        for t in range(K+1, n):
            ## Boostrapping
            s = len(Y)
            idx = np.random.choice(s,s)
            X_boot = X[idx,:]
            Y_boot = Y[idx,:]
            c_K = env.get_context(t, True)
            V_est = np.linalg.inv(np.matmul(np.transpose(X_boot), X_boot) + np.identity(d)*lam)
            beta_est = np.matmul(V_est, np.matmul(np.transpose(X_boot), Y_boot))
            mu_est = np.matmul(c_K, beta_est).reshape(K)
        
            ## pull arm
            a_t = np.argmax(mu_est) + 1
            r_t = env.pull(a_t, t)
            c_t = c_K[a_t - 1,:].reshape(d,1)
            A.append(a_t)
            R.append(r_t)
            Y = np.append(Y, np.array(r_t).reshape(1, 1), axis = 0)
            X = np.append(X, np.array(c_t).reshape(1, d), axis = 0)
            
            ## pseudo rewards
            for i in range(a):
                Y = np.append(Y, np.array(R_upper).reshape(1, 1), axis = 0)
                X = np.append(X, np.array(c_t).reshape(1, d), axis = 0)
                Y = np.append(Y, np.array(R_lower).reshape(1, 1), axis = 0)
                X = np.append(X, np.array(c_t).reshape(1, d), axis = 0)
            
            ## compute regret
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
        
        
    return R, A, regret

In [48]:
def Linear_PHE(env, a, lam, R_upper, R_lower, coefficient_sharing = True):
    '''
    Perturbed-history exploratio Algorithm
    This is a linear bandit algorithm
    ============================================
    INPUTS
        env: stochastic bandit environment
        a: Perturbation scale
        lam: regularization parameter
        R_upper: upper bound for reward  
        R_lower: lower bound for reward
        coefficient sharing: True if assuming true coefficient is same among arms
    ============================================
    OUPUTS 
        R: reward sequence, list with length n
        A is action sequence, list with length n
        regret: regret sequence, list with length n
    ''' 
    # set up
    n = env.n
    K = env.k
    d = env.d
    lam = lam + 1e-20
    regret = [0]
    A = []
    R = []
    
    if not coefficient_sharing:  #This is not correct
        beta_est = [np.zeros(d) for i in range(K)]
        V_est = [np.identity(d)*lam*(a + 1) for i in range(K)]
        Y_by_A = [np.empty((0,1))]*K
        X_by_A = [np.empty((0,d))]*K
        arm_count = np.zeros(K)

        # temporary liat/array
        mu_est = np.zeros(K)
        
        # pull each arm once
        for t in range(1, K+1):
            a_t = t
            c_t = env.get_context(t, False)
            r_t = env.pull(a_t, t)
            A.append(a_t)
            R.append(r_t)
            Y_by_A[a_t - 1] = np.append(Y_by_A[a_t - 1], np.array(r_t).reshape(1, 1), axis = 0)
            X_by_A[a_t - 1] = np.append(X_by_A[a_t - 1], np.array(c_t).reshape(1, d), axis = 0)
            V_est[a_t - 1] = V_est[a_t - 1] + (a + 1) * np.matmul(c_t, c_t.T)
            arm_count[a_t - 1] = arm_count[a_t  - 1] + 1
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
        
        # PHE loop
        for t in range(K+1, n):
            ## pertubed Histroy
            c_t = env.get_context(t, False)
            c_t = np.array(c_t).reshape(d, 1)
            U_sum = np.random.binomial(np.ceil(a * (t-1)).astype(int), 1/2)
            U = np.random.multinomial(U_sum, [1.0 / (t-1)]*(t-1), size = 1)
            Z_all = R_lower + (R_upper - R_lower) * U.reshape(t-1,1)
            start = int(0)
            for k in range(K):
                X = X_by_A[k]
                Y = Y_by_A[k]
                s = arm_count[k].astype(int)
                end = start + s
                end = int(end)
                Z = Z_all[start:end,0]
                Z = Z.reshape(s, 1)
                beta_est[k] = np.linalg.inv(V_est[k]) @ X.T @ (Y + Z)
                mu_est[k] = np.matmul(c_t.T, beta_est[k].reshape(d,1))
                start = int(start+arm_count[k])
'''
            for k in range(K):
                X = X_by_A[k]
                Y = Y_by_A[k]
                s = arm_count[k].astype(int)
                U_sum = np.random.binomial(np.ceil(a * s).astype(int), 1/2)
                U = np.random.multinomial(U_sum, [1.0/s]*s, size = 1)
                Z = R_lower + (R_upper - R_lower) * U.reshape(s,1)
                beta_est[k] = np.linalg.inv(V_est[k]) @ X.T @ (Y + Z)
                mu_est[k] = np.matmul(c_t.T, beta_est[k].reshape(d,1))
'''
            
            ## pull arm
            a_t = np.argmax(mu_est) + 1
            r_t = env.pull(a_t, t)
            A.append(a_t)
            R.append(r_t)
            Y_by_A[a_t - 1] = np.append(Y_by_A[a_t - 1], np.array(r_t).reshape(1, 1), axis = 0)
            X_by_A[a_t - 1] = np.append(X_by_A[a_t - 1], np.array(c_t).reshape(1, d), axis = 0)
            
            ## update
            V_est[a_t - 1] = V_est[a_t - 1] + (a + 1) * np.matmul(c_t, c_t.T)
            arm_count[a_t - 1] = arm_count[a_t  - 1] + 1
            
            ## compute regret
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)        
    
    else:    #This is correct
        beta_est = np.zeros(d)
        V_est = np.identity(d)*lam*(a + 1)
        Y = np.empty((0,1))
        X = np.empty((0,d))
        
        # pull each arm once
        for t in range(1, K+1):
            a_t = t
            c_K = env.get_context(t, True)
            c_t = c_K[a_t - 1,:].reshape(d,1)
            r_t = env.pull(a_t, t)
            A.append(a_t)
            R.append(r_t)
            Y = np.append(Y, np.array(r_t).reshape(1, 1), axis = 0)
            X = np.append(X, np.array(c_t).reshape(1, d), axis = 0)
            V_est = V_est + (a + 1) * np.matmul(c_t, c_t.T)
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
        
        # PHE loop
        for t in range(K+1, n):
            ## pertubed Histroy
            c_K = env.get_context(t, True)
            U_sum = np.random.binomial(np.ceil(a * (t-1)).astype(int), 1/2)
            U = np.random.multinomial(U_sum, [1.0 / (t-1)]*(t-1), size = 1)
            Z = R_lower + (R_upper - R_lower) * U.reshape(t-1,1)
            beta_est = np.linalg.inv(V_est) @ X.T @ (Y + Z)
            mu_est = np.matmul(c_K, beta_est).reshape(K)
        
            ## pull arm
            a_t = np.argmax(mu_est) + 1
            r_t = env.pull(a_t, t)
            c_t = c_K[a_t - 1,:].reshape(d,1)
            A.append(a_t)
            R.append(r_t)
            Y = np.append(Y, np.array(r_t).reshape(1, 1), axis = 0)
            X = np.append(X, np.array(c_t).reshape(1, d), axis = 0)
            
            ## update
            V_est = V_est + (a + 1) * np.matmul(c_t, c_t.T)
            
            ## compute regret
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
            
            
    return R, A, regret

In [49]:
def Linear_UCB(env, lam, delta = 0.05, coefficient_sharing = True):
    '''
    Upper Confidence Bound Algorithm
    This is a linear bandit algorithm
    ============================================
    INPUTS
        env: stochastic bandit environment
        lam: regularization parameter
        delta: tolarence, 1-delta is confidence level
        coefficient sharing: True if assuming true coefficient is same among arms
    ============================================
    OUPUTS 
        R: reward sequence, list with length n
        A is action sequence, list with length n
        regret: regret sequence, list with length n
    '''
    # set up
    n = env.n
    K = env.k
    d = env.d
    lam = lam + 1e-20
    regret = [0]
    A = []
    R = []
    
    if not coefficient_sharing:     # this is correct
        beta_est = [np.zeros(d) for i in range(K)]
        V_est = [np.identity(d)*lam for i in range(K)]
        Sigma_est = [np.identity(d)*(1/lam) for i in range(K)]
        g = [np.zeros(d).reshape(d,1) for i in range(K)]
        Y_by_A = [np.empty((0,1))]*K
        X_by_A = [np.empty((0,d))]*K

        # temporary liat/array
        ucb = np.zeros(K)
        radius = np.zeros(K)
        beta_norm_max = np.zeros(K)
        V_det = np.zeros(K)

        # pull each arm once
        for t in range(1, K+1):
            c_t = env.get_context(t, False)
            a_t = t
            r_t = env.pull(a_t, t)
            A.append(a_t)
            R.append(r_t)
            Y_by_A[a_t - 1] = np.append(Y_by_A[a_t - 1], np.array(r_t).reshape(1, 1), axis = 0)
            X_by_A[a_t - 1] = np.append(X_by_A[a_t - 1], np.array(c_t).reshape(1, d), axis = 0)
            V_est[a_t - 1] = V_est[a_t - 1] + np.matmul(c_t, c_t.T)
            Sigma_est[a_t - 1] = np.linalg.inv(V_est[a_t - 1])
            g[a_t - 1] = g[a_t - 1] + r_t*c_t.reshape(d,1)
            beta_est[a_t - 1] = Sigma_est[a_t - 1] @ g[a_t - 1]
            V_det[a_t - 1] = np.linalg.det(V_est[a_t - 1])
            beta_norm_tmp = np.linalg.norm(beta_est[a_t - 1])
            if beta_norm_tmp > beta_norm_max[a_t - 1]:
                beta_norm_max[a_t - 1] = beta_norm_tmp
            radius[a_t - 1] = np.sqrt(lam)*beta_norm_max[a_t - 1] + np.sqrt(2*np.log(1/delta) + np.log(V_det[a_t - 1]/lam**d))
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)

        # UCB loop
        for t in range(K+1, n):
            c_t = env.get_context(t, False)
            for k in range(K):
                ucb[k] = c_t.T @ beta_est[k] + radius[k] * np.sqrt(c_t.T @ Sigma_est[k] @ c_t)

            ## pull arm
            a_t = np.argmax(ucb) + 1
            r_t = env.pull(a_t, t)
            A.append(a_t)
            R.append(r_t)
            Y_by_A[a_t - 1] = np.append(Y_by_A[a_t - 1], np.array(r_t).reshape(1, 1), axis = 0)
            X_by_A[a_t - 1] = np.append(X_by_A[a_t - 1], np.array(c_t).reshape(1, d), axis = 0)

            ## least square update
            V_est[a_t - 1] = V_est[a_t - 1] + np.matmul(c_t, c_t.T)
            Sigma_est[a_t - 1] = np.linalg.inv(V_est[a_t - 1])
            g[a_t - 1] = g[a_t - 1] + r_t*c_t.reshape(d,1)
            beta_est[a_t - 1] = Sigma_est[a_t - 1] @ g[a_t - 1]

            ## confidence ellipsoid update 
            V_det[a_t - 1] = np.linalg.det(V_est[a_t - 1])
            beta_norm_tmp = np.linalg.norm(beta_est[a_t - 1])
            if beta_norm_tmp > beta_norm_max[a_t - 1]:
                beta_norm_max[a_t - 1] = beta_norm_tmp
            radius[a_t - 1] = np.sqrt(lam)*beta_norm_max[a_t - 1] + np.sqrt(2*np.log(1/delta) + np.log(V_det[a_t - 1]/lam**d))

            ## compute regret
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t) 
    
    else:     
        beta_est = np.zeros(d)
        V_est = np.identity(d)*lam
        Sigma_est = np.identity(d)*(1/lam)
        g = np.zeros(d).reshape(d,1)
        Y = np.empty((0,1))
        X = np.empty((0,d))
        
        # temporary liat/array
        ucb = np.zeros(K)
        radius = np.zeros(1)
        beta_norm_max = np.zeros(1)
        V_det = np.zeros(1)
        
        # pull each arm once
        for t in range(1, K+1):
            a_t = t
            c_K = env.get_context(t, True)
            c_t = c_K[a_t - 1,:].reshape(d,1)
            r_t = env.pull(a_t, t)
            A.append(a_t)
            R.append(r_t)
            Y = np.append(Y, np.array(r_t).reshape(1, 1), axis = 0)
            X = np.append(X, np.array(c_t).reshape(1, d), axis = 0)
            V_est = V_est + np.matmul(c_t, c_t.T)
            Sigma_est = np.linalg.inv(V_est)
            g = g + r_t*c_t.reshape(d,1)
            beta_est = Sigma_est @ g
            V_det = np.linalg.det(V_est)
            beta_norm_tmp = np.linalg.norm(beta_est)
            if beta_norm_tmp > beta_norm_max:
                beta_norm_max = beta_norm_tmp
            radius = np.sqrt(lam)*beta_norm_max + np.sqrt(2*np.log(1/delta) + np.log(V_det/lam**d))
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
        
        # UCB loop
        for t in range(K+1, n):
            c_K = env.get_context(t, True)
            for k in range(K):
                c_k = c_K[k,:].reshape(d,1)
                ucb[k] = c_k.T @ beta_est + radius * np.sqrt(c_k.T @ Sigma_est @ c_k)

            ## pull arm
            a_t = np.argmax(ucb) + 1
            r_t = env.pull(a_t, t)
            c_t = c_K[a_t - 1,:].reshape(d,1)
            A.append(a_t)
            R.append(r_t)
            Y = np.append(Y, np.array(r_t).reshape(1, 1), axis = 0)
            X = np.append(X, np.array(c_t).reshape(1, d), axis = 0)

            ## least square update
            V_est = V_est + np.matmul(c_t, c_t.T)
            Sigma_est = np.linalg.inv(V_est)
            g = g + r_t*c_t.reshape(d,1)
            beta_est = Sigma_est @ g

            ## confidence ellipsoid update 
            V_det = np.linalg.det(V_est)
            beta_norm_tmp = np.linalg.norm(beta_est)
            if beta_norm_tmp > beta_norm_max:
                beta_norm_max = beta_norm_tmp
            radius = np.sqrt(lam)*beta_norm_max + np.sqrt(2*np.log(1/delta) + np.log(V_det/lam**d))
            
            ## compute regret
            regret_t = regret[t - 1] + env.regret(a_t, t)
            regret.append(regret_t)
        
    return R, A, regret       

In [37]:
def Uniform_constrained_context(k, d, min = 0, max = 1):
    '''
    function to generate d-diemsnional context from uniform distribution such that
    norms are 1-(k-1)/k^2, 1-(k-2)/k^2, ..., 1-1/k^2, 1
    ============================================
    INPUT
        k: number of arms
        d: dimension of context, defualt is 2d context
        min: lower bound for uniform distribution, default is 0
        max: upper bound for uniform distribution, default is 1
    ============================================
    OUPUT
        c: contexts for all arms, k by d numpy array
    '''
    c = np.zeros(k*d).reshape(k,d)
    for i in range(k):
        c_k = np.random.uniform(low = min, high = max, size = d).reshape(d) 
        norm = np.linalg.norm(c_k)
        c_k = c_k / norm
        c[i,:] = c_k
    return c

In [15]:
horizon_len = 1000
k = 100
d = 5
beta = np.random.uniform(-0.5, 0.5, 1*d).reshape(1, d)
beta = beta/np.linalg.norm(beta)
sigma_seq = np.ones(k) * np.sqrt(0.1)
nu = [np.random.uniform(0, 1, 1*d).reshape(d)]*k

In [16]:
def Gaussian_constrained_context(k, d, mean = nu):
    '''
    function to generate d-diemsnional context from Gaussian distribution
    such that norms are 1
    ============================================
    INPUT
        k: number of arms
        d: dimension of context, defualt is 2d context
        mean: mean for d-dimensional Gasussian
    ============================================
    OUPUT
        c: contexts for all arms, k by d numpy array
    '''
    cov = [1/(2*k)*np.identity(d)]*k
    c = np.zeros(k*d).reshape(k,d)
    for i in range(k):
        c_k = np.array(stats.multivariate_normal.rvs(mean = mean[i], cov = cov[i], size = 1)).reshape(d) 
        norm = np.linalg.norm(c_k)
        c_k = c_k / norm
        c[i,:] = c_k
    return c

In [17]:
env = Linear_Bandit(k = k, n = horizon_len, beta = beta, sigma = sigma_seq, random_context = True, 
                    gen_context = Gaussian_constrained_context)

In [86]:
def beta_design(k, d):
    '''
    function to beta for linear bandit with covariates setting
    ============================================
    INPUT
        k: number of arms
        d: dimension of context
    ============================================
    OUPUT
        beta: true parameter, k by d
    '''
    beta = np.ones(k*d).reshape(k,d)
    idx = []
    for kk in range(k):
        b = np.random.binomial(d, 1/2)
        idx_k = np.random.choice(d, b, replace = False)
        beta[kk,:][idx_k] = -1
        beta[kk,:] = beta[kk,:] + np.random.uniform(low = -0.95, high = 0.95, size = d)
        beta[kk,:] = (kk + 1)/k * beta[kk,:]/np.linalg.norm(beta[kk,:])
    
    return beta